In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from glob import glob

In [4]:
import itertools

In [5]:
import pygsheets as pyg

In [6]:
from apiclient import discovery

In [7]:
from oauth2client.file import Storage

In [8]:
import httplib2

In [9]:
import time

#### some pandas opts

In [10]:
pd.options.display.max_columns = 100

In [11]:
pd.options.display.max_rows = 500

#### secret file for auth for anything needed in this notebook

In [12]:
secret_json = "./client_secret_986841527205-ndf6fbsfmrhjinnofm47j6olvmm11smn.apps.googleusercontent.com.json"

#### auth for drive. quick start script that takes secret client json file and creates conctents of
#### ```~/.credentials```
#### is needed!!!
#### it will NOT work inside the notebook since here the argparse Namespace is broken! it is needed for the flags var (see quickstart)

In [13]:
credential_path = "/home/dizak/.credentials/drive-python-quickstart.json"

In [14]:
store = Storage(credential_path)

In [15]:
credentials = store.get()

In [16]:
http = credentials.authorize(httplib2.Http())

In [17]:
service = discovery.build("drive", "v3", http=http)

#### mind if len(results["files"]) > 1000 which is max per page you have to take care of pageToken.
#### pageToken can be retrieved by 
#### ```res = service.files().list(pageSize=1000, fields="nextPageToken, files").execute()```
#### ```token = res["nextPageToken"]```
#### and then passed as arg like:
#### ```service.files().list(pageSize=1000, pageToken=token, fields="nextPageToken, files").execute()```

In [18]:
results = service.files().list(pageSize=1000).execute()

In [19]:
len(results["files"])

507

#### authorize pyg and set retry limit

In [20]:
gc = pyg.authorize(outh_file=secret_json,
                   outh_nonlocal=True,
                   retries=5)

#### title for the google sheet and xls

In [21]:
sheet_name = "BY4741-WT-day70"

#### xls output directory

In [22]:
xls_out_dir = "/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-WT-day70/"

#### gdrive output directory

In [23]:
gdrive_out_dir = [i["id"] for i in results["files"] if i["name"] == "testing"][0]

In [24]:
gdrive_out_dir

u'0BwDD9bCCIcilTkMwNGNjWl9vVXc'

#### naming of the sampling levels. it is INDEPENDENT of the actual naming in the CSVs

In [25]:
sampling_levels = ["N1", "N2", "N3"]

#### columns names for by which sorting will be done

In [26]:
sort_vals = ["Minimum"]

#### CSVs paths

In [27]:
strain_gene_day_N1_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/WT-day70/N1/*csv")

In [28]:
strain_gene_day_N2_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-WT-day70/BY-WT2-day70/*csv")

In [29]:
strain_gene_day_N3_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-WT-day70/BY-WT3-day70/*csv")

#### get non-redundant list of genes in the inputfiles

In [30]:
def find_flat_value(inputfiles_list,
                    col_name = "CDS"):
    """
    Get flat list of desired values from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    col_name: str
        Desired column name in the input CSV file.
    
    Returns
    -------
    list of desired values.
    """
    values_list = []
    for i in inputfiles_list:
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        elif col_name not in df.columns:
            pass
        else:
            values_list.append(df[col_name].dropna().drop_duplicates().tolist())
    return list(itertools.chain.from_iterable(values_list))

#### get values from input files by key

In [31]:
def find_by_key(inputfiles_list,
                key):
    """
    Get pandas.DataFrame selected by a given key from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    key: str, int, float, bool
        Key used as query against rows in the CSV files.
    value_col: str
        Column name which holds values to be returned.
    
    Returns
    -------
    dict of lists of desired values if pandas.Dataframe not empty
    None if pandas.DataFrame empty
    """
    values_list = []
    for i in inputfiles_list:
        filename = "".join(i.split("/")[-1].split(".")[:-1])
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        else:
            if isinstance(key, str) == True:
                df_dtype_sel = df.select_dtypes(include=["object"])
            elif isinstance(key, int) == True:
                df_dtype_sel = df.select_dtypes(include=["int"])
            elif isinstance(key, float) == True:
                df_dtype_sel = df.select_dtypes(include=["float"])
            elif isinstance(key, bool) == True:
                df_dtype_sel = df.select_dtypes(include=["bool"])
            else:
                raise ValueError("key must str, int, float or bool dtype")
            for col in df_dtype_sel.columns:
                df_sel = df[df_dtype_sel[col] == key]
                if len(df_sel) > 0:
                    return {"dataframe": df_sel,
                            "filename": filename}

#### get whole set of pandas.DataFrames selections in one dict

In [32]:
def get_dfs_set(key_list,
                files_list,
                vals=["Minimum",
                      "Maximum",
                      "Change"],
                df_key="dataframe",
                key_index_name="Gene",
                smpl_index_name="Sample",
                row_index_name="Number",
                index_by_key=True,
                smpl_index_val=None,):
    """
    Get desired values in pandas.Dataframe gathered in dict by
    the list of keys.
    
    Parameters
    -------
    key_list: list, tuple
        List of keys by which pandas.Dataframes are
        initially selected.
    files_list: list, tuple
        List of input CSV files.
    vals: list of str, default: ["Minimum", 
                                 "Maximum",
                                 "Change"]
        Columns names holding data in pandas.DataFrames
        selected.
    df_key: str, default: <"dataframe">
        Key for generic pandas.DataFrame selection for
        SNPs-sheets_merge.find_by_key function.
    key_index_name: str, default: <"Gene">
        Name for index of selection key.
    smpl_index_name: str, default: <"Sample">
        Name for index of sample.
    row_index_name: str, default: <"Number">
        Name for numeric row index.
    index_by_key: bool, default: True
        Enables multiindexing.
    smpl_index_val: str, default: <None>
        Adds sample name to multiindexing if not <None>
    """
    out_dict = {}
    for i in key_list:
        key_vals = find_by_key(files_list,
                               key=i)[df_key]
        out_dict[i] = key_vals[vals]
    if index_by_key is True:
        for i in out_dict:
            key_index = [i] * len(out_dict[i])
            if smpl_index_val is not None:
                smpl_index = [smpl_index_val] * len(out_dict[i])
                tpls = list(zip(*[key_index,
                                  smpl_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          smpl_index_name,
                                                          row_index_name])
            else:
                tpls = list(zip(*[key_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          row_index_name])
            out_dict[i].index = mindex
    return out_dict

#### merge any given number of dfs

In [33]:
def merge_dfs(dfs,
              sort_cols=["Minimum",
                         "Maximum"],
              reconstr_index=True):
    """
    Merge any number of pandas.DataFrame into one.
    Indexes must be identical in all the pandas.DataFrames.
    
    Parameters
    -------
    dfs: list
        list of pandas.DataFrames to merge.
    sort_cols: list, None
        list of col names to sort the final
        pandas.DataFrame by. No sorting if None.
    reconstr_index: bool
        Copy index from the original pandas.DataFrames
        to the final one.
    """
    for x in [set(i.index) for i in dfs]:
        assert len(x) == 1, "Indices are not homogenic."
    new_index = list(x)
    df = reduce(lambda df1, df2: pd.merge(left=df1,
                                          right=df2,
                                          how="outer"),
                dfs)
    if sort_cols is not None:
        df.sort(columns=sort_cols)
    if reconstr_index is True:
        df.index = len(df) * new_index
    return df

#### let's find out which CDS are present in all the files one by one

In [34]:
strain_gene_day_N1_CDSs = find_flat_value(strain_gene_day_N1_files)
strain_gene_day_N2_CDSs = find_flat_value(strain_gene_day_N2_files)
strain_gene_day_N3_CDSs = find_flat_value(strain_gene_day_N3_files)

#### let's gather some info about each of the CDS from each sample

In [35]:
des_cols = ["Change",
            "Protein Effect",
            "Polymorphism Type",
            "Variant Frequency",
            "Average Quality",
            "Minimum",
            "Maximum"]

In [36]:
strain_gene_day_N1_dfs = get_dfs_set(strain_gene_day_N1_CDSs,
                                     strain_gene_day_N1_files,
                                     vals=des_cols,
                                     smpl_index_val=sampling_levels[0])
strain_gene_day_N2_dfs = get_dfs_set(strain_gene_day_N2_CDSs,
                                     strain_gene_day_N2_files,
                                     vals=des_cols,
                                     smpl_index_val=sampling_levels[1])
strain_gene_day_N3_dfs = get_dfs_set(strain_gene_day_N3_CDSs,
                                     strain_gene_day_N3_files,
                                     vals=des_cols,
                                     smpl_index_val=sampling_levels[2])

#### let's unwind them all from this dict into lists

In [37]:
N1_dfs = [strain_gene_day_N1_dfs[i] for i in strain_gene_day_N1_dfs.keys()]
N2_dfs = [strain_gene_day_N2_dfs[i] for i in strain_gene_day_N2_dfs.keys()]
N3_dfs = [strain_gene_day_N3_dfs[i] for i in strain_gene_day_N3_dfs.keys()]

#### let's merge them to see changes between samples.
#### rememeber now cannot use the merge_dfs function since multindex is NOT homogenic

In [38]:
N1_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N1_dfs)

In [39]:
N2_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N2_dfs)

In [40]:
N3_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N3_dfs)

In [41]:
N1_N2_N3_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
                     [N1_df, N2_df, N3_df])

In [42]:
N1_N2_N3_df.sort_values(sort_vals, inplace=True)

#### let's make xls

In [43]:
writer = pd.ExcelWriter("{}/{}.xls".format(xls_out_dir, sheet_name))

In [44]:
for i in set(strain_gene_day_N1_CDSs +
             strain_gene_day_N2_CDSs +
             strain_gene_day_N3_CDSs):
    N1_N2_N3_df.xs(i).to_excel(excel_writer=writer,
                               sheet_name=i)

In [45]:
writer.save()

#### let's make sheet on gdrive

In [46]:
sh = gc.create(sheet_name, parent_id=gdrive_out_dir)

In [47]:
for i in set(strain_gene_day_N1_CDSs +
             strain_gene_day_N2_CDSs +
             strain_gene_day_N3_CDSs):
    time.sleep(3)
    wks = sh.add_worksheet(i)
    wks.set_dataframe(N1_N2_N3_df.xs(i),
                      (1, 1),
                      copy_index=True,
                      fit=True)
sh.del_worksheet(sh.worksheet_by_title("Sheet1"))